# **Feature Engineering**

This notebook applies feature engineering techniques such as rolling window statistics, normalization, and dimensionality reduction (PCA). These transformations help capture degradation patterns while reducing noise in sensor data.

In [ ]:
%run ./01_eda.ipynb # Run all the code in the previous notebook

Exception: File `'./01_eda.ipynb.py'` not found.

**Rolling Feature Engineering**

We create new features that summarize sensor and operational setting behavior over time.

Instead of using raw values at each cycle, rolling statistics capture *trends* (slopes, averages, variability).

This helps the model detect gradual degradation patterns that are predictive of Remaining Useful Life (RUL).

In [ ]:
# Make a copy of the dataset so we can safely add new rolling features
rolling_data = data.copy()

# 1. Rolling Standard Deviation (window=5 cycles)
#    - Std shows variability over the window, which can indicate instability or early fault signals.rolling_data[op_cols + sensor_cols] = rolling_data.groupby("unit_number")[op_cols + sensor_cols].rolling(window=5, min_periods=1).mean().reset_index(level=0, drop=True)
rolling_data[[f'{col}_std' for col in op_cols + sensor_cols]] = (
    rolling_data.groupby("unit_number")[op_cols + sensor_cols]
    .rolling(window=5, min_periods=1).std()
    .reset_index(level=0, drop=True)
)

# 2. Rolling Slope (window=5 cycles)
#    - Measures the *rate of change* of each sensor signal.
#    - Positive slope = increasing trend (e.g., rising temperature or vibration).
#    - Negative slope = decreasing trend (e.g., dropping pressure).
#    - This can reveal whether an engine is degrading quickly or stabilizing.
for col in op_cols + sensor_cols:
    rolling_data[f'{col}_slope'] = rolling_data.groupby("unit_number")[col].diff(periods=5).rolling(window=5, min_periods=1).mean().reset_index(level=0, drop=True)

# 3. Rolling Variance (window=20 cycles)
#    - Captures longer-term fluctuations in sensor signals.
#    - High variance can signal abnormal oscillations or instability in the engine
for col in op_cols + sensor_cols:
    rolling_data[f'{col}_var'] = rolling_data.groupby("unit_number")[col].rolling(window=20, min_periods=1).var().reset_index(level=0, drop=True)

# 4. Normalized Cycle Index
#    - Each engine runs for a different number of cycles before failure.
#    - Normalized cycle = (current cycle / engine's max cycle).
#    - This gives the model a relative measure of how far along the engine is in its lifecycle.
max_cycles = data.groupby("unit_number")["time_in_cycles"].max().reset_index()
max_cycles.columns = ["unit_number", "max_cycles"]

rolling_data = rolling_data.merge(max_cycles, on="unit_number", how="left")
rolling_data["normalized_cycle"] = rolling_data["time_in_cycles"] / rolling_data["max_cycles"]

# Drop helper column since it's no longer needed
rolling_data = rolling_data.drop(columns=["max_cycles"])

rolling_data

NameError: name 'data' is not defined

**PCA (Principal Component Analysis)**

PCA is a dimensionality reduction technique.

It transforms correlated features (like sensors and operational settings).


***Why use PCA here?***


*  Many sensors are strongly correlated (e.g temperature/pressure pairs).
*  High correlation adds redundancy and noise, making it harder for the models to generalize.
*  PCA reduces thi redundancy by keeping the main variance directions.
*  We keep only the top components that explain most of the variance, which reduces dimensionality while preserving useful signal.

***Steps:***


1.   Standardize features (zero mean, unit variance), which ensures all sensors are on the same scale.
2.   Apply PCA to extract 10 principal components.
3.   Store components as new features (PC1, PC2, ... ,PC10) for use in training instead of (or alongside) original sensors

This helps improve training efficiency and may reduce overfitting

In [ ]:
# Step 1: Standardize the operational + sensor data
standardized_data = StandardScaler().fit_transform(rolling_data[op_cols+sensor_cols])

# Print first 20 rows to verify standardized values
print(pd.DataFrame(standardized_data, columns=op_cols + sensor_cols).head(20))

# Step 2: Run PCA to extract top 10 components
pca = PCA(n_components=10)
X_pca = pca.fit_transform(standardized_data)

# Print summary: number of components and their explained variance ratio
# Explained variance ratio = % of dataset variance captured by each component
print(f"Number of components selected: {pca.n_components_}")
print(pca.explained_variance_ratio_)

# Step 3: Create a DataFrame for the PCA features (PC1–PC10)
X_pca_df = pd.DataFrame(X_pca, columns=[f"PC{i+1}" for i in range(X_pca.shape[1])])

# Print first 20 rows of transformed data
print("\n\n")
print(X_pca_df.head(20))

**Final Dataset (combined with the PCA data)**

In [ ]:
final_data = rolling_data.reset_index(drop=True).join(X_pca_df) # Join the pca data with rolling data

display(final_data.head(30))